# Optimization notebook
This notebook is to run a multi-objective optimization with the Zambezi model. Two alternatives will be given to that end. First implementation is with the Platypus library which has a fairly simple interface. The second implementation is the EMA Workbench which additionally offers parallelization functionality

In [12]:
import pandas as pd
import numpy as np
import os
os.chdir('../src')
from model_zambezi_OPT import model_zambezi

from platypus import NSGAII, Problem, Real

In [13]:
ZambeziProblem = model_zambezi()

## Platypus (NSGAII)

In [ ]:
problem = Problem(ZambeziProblem.Nvar, ZambeziProblem.Nobj)
problem.types[:] = Real(0, 1)
problem.function = ZambeziProblem.evaluate

algorithm = NSGAII(problem=problem, population_size=20)
algorithm.run(100)

In [ ]:
objectives_outcome = dict()
for i, column_name in enumerate(['Hydropower','Environment','Irrigation']):
    objectives_outcome[column_name] = [s.objectives[i] for s in algorithm.result]

objectives_df = pd.DataFrame(objectives_outcome)


In [ ]:
from various_plots import parallel_plots
parallel_plots(objectives_df)

## EMA Workbench

In [8]:
from model_specification_ema import model_wrapper
from ema_workbench import (RealParameter, ScalarOutcome, Constant,
                           Model)

model = Model('zambeziproblem', function=model_wrapper)

model.levers = [RealParameter('v' + str(i), -1, 1) for i in range(ZambeziProblem.Nvar)]

#specify outcomes
model.outcomes = [ScalarOutcome('Hydropower', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Environment', ScalarOutcome.MINIMIZE),
                  ScalarOutcome('Irrigation', ScalarOutcome.MINIMIZE)]

In [11]:

from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, ema_logging
import dill

ema_logging.log_to_stderr(ema_logging.INFO)

with SequentialEvaluator(model) as evaluator:
    results = evaluator.optimize(nfe=100,  searchover='levers', epsilons=[0.1, 0.1, 0.1])




  0%|                                                  | 0/100 [00:00<?, ?it/s][MainProcess/ERROR] name 'ZambeziProblem' is not defined
Traceback (most recent call last):
  File "C:\Users\whitl\anaconda3\lib\site-packages\ema_workbench\em_framework\experiment_runner.py", line 92, in run_experiment
    model.run_model(scenario, policy)
  File "C:\Users\whitl\anaconda3\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\whitl\anaconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 350, in run_model
    outputs = self.run_experiment(experiment)
  File "C:\Users\whitl\anaconda3\lib\site-packages\ema_workbench\util\ema_logging.py", line 152, in wrapper
    res = func(*args, **kwargs)
  File "C:\Users\whitl\anaconda3\lib\site-packages\ema_workbench\em_framework\model.py", line 403, in run_experiment
    model_output = self.function(**experiment)
  File "C:\Users\whitl\OneDrive\Documenten\MASTER\Year 2\THE

EMAError: exception in run_model
Caused by: NameError: name 'ZambeziProblem' is not defined

In [ ]:
from ema_workbench import MultiprocessingEvaluator, SequentialEvaluator, IpyparallelEvaluator, ema_logging
from ema_workbench.em_framework.ema_ipyparallel import (start_logwatcher, set_engine_logger,
                                  initialize_engines, cleanup, _run_experiment)
import dill
#from ipyparallel import Client


#rc = Client()

ema_logging.log_to_stderr(ema_logging.INFO)

with MultiprocessingEvaluator(model) as evaluator:
   # results = evaluator.optimize(nfe=1, searchover='levers',
   results = evaluator.optimize(nfe=1,
                                epsilons=[0.5,]*len(model.outcomes))

[MainProcess/INFO] pool started with 8 workers
  0%|                                                    | 0/1 [00:00<?, ?it/s]